# Co and Cu speciation in hydrothermal fluids

In this notebook, I investigate the aqueous speciation of Cu and Co in acidic, saline hydrothermal fluids. 

In [19]:
import thermofun as fun
import thermohubclient as client
import reaktoro as rkt
import math as math

In [20]:
database = fun.Database('/Users/robertcollar/Documents/Github/thermofun-jupyter/databases/aq17-gem-lma-thermofun.json')
engine = fun.ThermoEngine(database)

In [21]:
reaction_properties = engine.thermoPropertiesReaction(298.15, 1e5, "CoCl4-- = Co++ + 4Cl-") #[K, Pa]
logK = reaction_properties.log_equilibrium_constant

RuntimeError: 
******************************************************************************
*** Error: Cannot get an instance of the substance `Cl-` in the database.
*** Reason: There is no such substance in the database.
*** Location: :158
******************************************************************************



Chemical Reaction: $$Al_2(SO_{4}^{2-})_{3(s)} \leftrightarrow 2Al^{3+}_{(aq)} + 3SO_{4(aq)}^{2-}$$

In [37]:
# list of species in our reaction
species = ["Aluminum Sulfate","Al+3", "SO4-2"]

# stoichiometric coefficients
st_coef = [-1, 2, 3]
print(type(st_coef))

# temperature and pressure conditions
T = 298.15 # [K]
P = 1e5 # [Pa]

# gas constant
R = 8.314 # [J*mol^-1*K^-1]

<class 'list'>


In [44]:
# initialize thermodynamic properties lists
G = []
H = []
S = []
V = []
Cp = []

In [45]:
for i, specie in enumerate(species): 
    properties = engine.thermoPropertiesSubstance(T, P, specie) # note that "Aluminum Sulphate" corresponds with the "symbol," not the "name," in the json file. 
    G.append(properties.gibbs_energy.val)
    H.append(properties.enthalpy.val)
    S.append(properties.entropy.val)
    V.append(properties.volume.val) # J/bar
    Cp.append(properties.heat_capacity_cp.val)
    
print(G)


[-3099919.0, -486594.00000420073, -744459.1200048233]


In order to calculate the solubility product of this reaction, we will first need calculate its $\Delta_r G°$.

In [40]:
delGr = sum(a * b for a, b in zip(st_coef, G))
print(delGr)

[3099919.0, -973188.0000084015, -2233377.36001447]
-106646.36002287129


Now, we can calculate $K_{sp}$ using $\Delta_rG° = -RTln(K_{sp})$

In [41]:
Ksp = math.exp(-delGr/(R*T))
print(Ksp)

4.838085621773235e+18


In [42]:
print(f'gibbs_energy = {G} J/mol')
print(f'enthalpy = {H} J/mol')
print(f'entropy = {S} J/mol/K')
print(f'volume = {V} J/bar')
print(f'heat_capacity_cp = {Cp} J/mol/K')

gibbs_energy = [-3099919.0, -486594.00000420073, -744459.1200048233] J/mol
enthalpy = [-3465052.0, -537882.7611236677, -909602.4738909043] J/mol
entropy = [239.36, -339.74254304157995, 18.825068971722065] J/mol/K
volume = [10.131999969482, -4.534256034956643, 1.2912082818768043] J/bar
heat_capacity_cp = [276.7202772486522, -119.51466114943673, -266.28618610568867] J/mol/K
